DMIA [Спорт 2018 весна] hw0

33 of 115 /  Oleg Cheremisin / Private LB 0.91229

TLDR: Upsampling + Tfidf + SGD


Попробовано:
1. апсемплинг - увеличивает результат на тесте и на лидерборде (поднялся на 5 позиций), применен
2. pymorphy2 нормализация - ухудшает результат и на тесте и на лидерборде 0.89782 > 0.84251, убран
3. xgboost с заданным по расбалансу классов scale_pos_weight - скор лб чуть ниже чем sgd upscale 0.87976 vs 0.89782 
4. 5 последних букв вместо слова целином n-gramm ами - ухудшает, убран

In [8]:
import pandas as pd
import numpy as np

random_state = 42

from sklearn.utils import resample

train = pd.read_csv('datasets/dmia-surnames/train.csv')
test = pd.read_csv('datasets/dmia-surnames/test.csv')
sample_submission = pd.read_csv('datasets/dmia-surnames/sample_submission .csv')

In [9]:
train_majority = train[train['Label']==0]
train_minority = train[train['Label']==1]
train_minority_upsampled =  resample(train_minority, 
                                 replace=True,
                                 n_samples=train_majority.shape[0], 
                                 random_state=random_state) 

train = train_majority.append(train_minority_upsampled)

train = train.sample(frac=1)
train = train.reset_index(drop=False).drop(["index"], axis=1)

In [10]:
train['Word'] = train['Word'].apply(lambda x: x.lower()) #.apply(lambda x: x[-5:])
test['word'] = test['Word'].apply(lambda x: x.lower()) #.apply(lambda x: x[-5:])

train.sample(15)

,Word,Label
89577,кимбл,1
149359,константиновым,1
90433,меркадо,1
175098,монетой,0
109695,граффом,1
175377,тосканини,1
64354,пендерецкого,1
152841,порше,1
39567,биологи,0
153916,глотов,1


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 5), analyzer='char_wb')
tfidf.fit(train.append(test)['Word'])

TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
ttrain = tfidf.transform(train['Word'])
ttest = tfidf.transform(test['Word'])

In [13]:
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor

X_train, X_test, y_train, y_test = train_test_split(ttrain, train['Label'], 
    test_size=0.2,random_state = random_state )

model = SGDClassifier()

parameters = {
    'max_iter': [300],
    'loss':['modified_huber']
}

grid = GridSearchCV(model,parameters,scoring='roc_auc')

grid.fit(X_train, y_train)

print(grid.best_score_)
print(grid.best_estimator_)

0.968553629048
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', max_iter=300,
       n_iter=None, n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)


In [14]:
submission = sample_submission
submission['Prediction'] = grid.best_estimator_.predict(ttest)
submission.to_csv('datasets/dmia-surnames/submission_2018-02-17.csv', index=False)